## Imports & Data Upload

In [2]:
import spacy
from spacy import displacy
import pandas as pd
from spacy.matcher import Matcher, DependencyMatcher
from collections import Counter

from markdown import Markdown
from io import StringIO



In [3]:
df = pd.read_csv("/Users/finnmacken/Desktop/Code/ChatStructure/hepta-feature-requests.csv")


## Pre-Processing

### Convert All Content to Strings

In [4]:
df['content'] = df['content'].astype(str)

#### Remove Markdown Syntax

In [5]:
def unmark_element(element, stream=None):
    if stream is None:
        stream = StringIO()
    if element.text:
        stream.write(element.text)
    for sub in element:
        unmark_element(sub, stream)
    if element.tail:
        stream.write(element.tail)
    return stream.getvalue()


# patching Markdown
Markdown.output_formats["plain"] = unmark_element
__md = Markdown(output_format="plain")
__md.stripTopLevelTags = False


def unmark(text):
    return __md.convert(text)

In [6]:
df['content'] = df['content'].apply(unmark)

In [7]:
df

,content,created_at,author,reference,embeds,mentions,id,attachments
0,!history,2022-09-13 10:22:31.756000+00:00,finn,NaN,[],[],1019191342687072276,[]
1,Thanks for the feedback! I'll discuss it with ...,2022-09-13 02:42:46.824000+00:00,Alan Chan,<MessageReference message_id=10190620301766164...,[],[<User id=174103652556996608 name='robotic_sca...,1019075643096432781,[]
2,a couple feature requests:\n\n\nit would be fa...,2022-09-13 01:48:41.251000+00:00,robotic_scarab,NaN,[],[],1019062030176616458,[<Attachment id=1019062029572649100 filename='...
3,"Yep, when letting go of the connection arrow i...",2022-09-12 21:54:42.818000+00:00,maxlinworm,<MessageReference message_id=10189863575924450...,[],[<User id=710847265581039616 name='Sams_Here' ...,1019003148720951346,[]
4,"If I may clarify, you want to create a new car...",2022-09-12 20:47:59.501000+00:00,Sams_Here,<MessageReference message_id=10189826926557266...,[],[<User id=776122030230732820 name='maxlinworm'...,1018986357592445019,[]
...,...,...,...,...,...,...,...,...
2667,Ability to choose/assign colors to hashtags,2021-06-20 15:31:06.111000+00:00,Sams_Here,NaN,[],[],856194458000687144,[]
2668,"Hover over Graph View Icon shows ""graph view""",2021-06-20 14:33:36.692000+00:00,Sams_Here,NaN,[],[],856179990089564201,[]
2669,Ctrl + Scroll Wheel = Zoom graph view,2021-06-20 14:33:01.968000+00:00,Sams_Here,NaN,[],[],856179844446552094,[]
2670,Type Tool\nA button somewhere on the board wit...,2021-06-20 11:43:34.928000+00:00,Vincer,NaN,[],[],856137200789880894,[]


## Pattern Declaration & Testing

In [8]:
nlp = spacy.load("en_core_web_sm")
matcher = Matcher(nlp.vocab)

In [9]:
auxBe = [
    # this might be useful
    # this could be [a] [very] helpful [feature]
    [{"POS": "AUX"}, {"POS": "ADV", "OP": "*"}, {"LOWER": "be"}, {"POS": "DET"}, {"POS": "ADV", "OP": "*"}, {"LOWER": {"IN": ["great", "excellent", "amazing", "nice", "good", "awesome", 
                                                                                       "helpful", "useful", "terrific", "cool", 'like', 'love', 'appreciate', 'ideal', 'convenient'
                                                                                      ]}}], 
]

auxDesire = [
    [{"POS": "AUX", "TAG": {"IN": ["MD", "RB"]}}, {"POS": "ADV", "OP": "*"}, {"LOWER": {"IN": ['like', 'love', 'appreciate', 'want']}}]
]

pronounDesire = [
    [{"POS": "PRON"}, {"LOWER": {"IN": ["really", "desperately", "just", "only", "often", "actually"]}, "OP": "*"}, {"LOWER": {"IN": ['want', 'wish']}} ]
]

howAbout = [
    [{"LOWER": "how"}, {"LOWER": 'about'}]
]

couldPronoun = [
    # could we [also] do/be
    [{"LOWER": {"IN": ['could', 'can']}}, {"POS": "PRON"}, {"POS": "ADV", "OP": "*"}, {"POS": {"IN": ["VERB", "AUX"]}}]
]

pronounCould = [
    # we could [also] do/be
    # which could be
    # we can improve
  [{"POS": {"IN": ['you', 'we']}}, {"LOWER": {"IN": ['could']}}, {"POS": "ADV", "OP": "*"}, {"POS": {"IN": ["VERB", "AUX"]}}]  
]

auxPossibility = [ 
    # would it be possible
    # is it possible
    [{"POS": "AUX"}, {"POS": "PRON"}, {"POS": "AUX", "OP": "*"}, {"POS": "DET", "OP": "*"}, {"LOWER": {"IN": ['possible', 'possibility', 'option']}}]
]

possibilityVerb = [
    [{"LOWER": {"IN": ['possible', 'possibility']}}, {"POS": {"IN": ["PART", "ADP"]}}, {"POS": "VERB"}]
]

possibilityAdposition = [
    # possibility to
    [{"LOWER": {"IN": ['possible', 'possibility']}}, {"POS": "ADP"}]
]


noun_suggest_list = ['suggestion', 'idea', 'proposal', 'recommendation', 'submission', 'advice', 'proposition']

pronounSuggest = [
    # i suggest
    [{"POS": "PRON"}, {"LOWER": {'IN': ['suggest', 'propose', 'recommend', 'advocate', 'move', 'encourage']}}]
]

pronounSuggestionAux = [
    # my suggestion is
    # the idea is
    [{"POS": {"IN": ["PRON", "DET"]}}, {"LEMMA": {"IN": noun_suggest_list}}, {"POS": "AUX"}]
]

detNounSuggest = [
    # a couple of ideas
    # several suggestions
   [{"POS": "DET", "OP": "*"}, {"POS": {"IN": ['NOUN', "ADJ"]}}, {"POS": "ADP", "OP": "*"}, {"LEMMA": {"IN": noun_suggest_list}}]
]


shouldVerb = [
    # should [absolutely] be allowed to
    # should [absolutely] be created
    [{"LOWER": "should"}, {"POS": "ADV", "OP": "*"}, {"POS": "AUX", "OP": "*"}, {"POS": {"IN": ["VERB", "ADJ"]}}]
]

featureRequestPunct = [
    [{"LOWER": {"IN": ['feature', 'request', 'suggestion']}}, {"LOWER": {"IN": [":", 'for']}}]
]

hopeRequest = [
    [{"LOWER": "hope"}, {"LOWER": {"IN": ['you', 'that', 'we', 'there']}}]
]

whatIf = [
    [{"LOWER": "what"}, {"LOWER": 'if'}]
]

plannedTo = [
    [{"LOWER": {"IN": ["is", 'do', 'are']}}, {"POS": "PRON"}, {"POS": "DET", "OP": "*"}, {"LEMMA": "plan"}, {"POS": {"IN": ['PART', 'ADP']}}]
]

wayTo = [
    [{"LOWER": "is"}, {"LOWER": 'there'}, {"LOWER": "a"}, {"LOWER": 'way'}, {"LOWER": "to"}]
]


matcher.add("AuxBe", auxBe)

matcher.add("AuxDesire", auxDesire)
matcher.add("PronounDesire", pronounDesire)

matcher.add("HowAbout", howAbout)

matcher.add("CouldPronoun", couldPronoun)
matcher.add("PronounCould", pronounCould)

matcher.add("AuxPossibility", auxPossibility)
matcher.add("PossibilityVerb", possibilityVerb)
matcher.add("PossibilityAdposition", possibilityAdposition)

matcher.add("PronounSuggest", pronounSuggest)
matcher.add("PronounSuggestionAux", pronounSuggestionAux)
matcher.add("DetNounSuggest", detNounSuggest)

matcher.add("ShouldVerb", shouldVerb)

matcher.add("FeatureRequestPunct", featureRequestPunct)

matcher.add("HopeRequest", hopeRequest)

matcher.add("WhatIf", whatIf)

matcher.add("PlannedTo", plannedTo)
matcher.add("WayTo", wayTo)


In [10]:
def test_string(test_str):
    doc = nlp(test_str)
    matches = matcher(doc)
    if len(matches) != 0:
        print('\033[92m' + f"Match Found for: {test_str}" + '\033[1m' + " ( " + ", ".join([nlp.vocab.strings[i[0]] for i in matches]) + " )" +  '\033[0m' + '\033[0m')
        for idx, match in enumerate(matches):
            print(f"Match {idx+1}: {doc[match[1]: match[2]]} ({match[1]}, {match[2]})")
    else:
        print('\033[91m' + f"No match found for: {test_str}" + '\033[0m') 

In [11]:
def test_doc(doc_index, doc):
    matches = matcher(doc)
    if len(matches) != 0:
        print('\033[92m' + f"[{doc_index+1}] Match Found for: {doc.text}" + '\033[1m' + " ( " + ", ".join([nlp.vocab.strings[i[0]] for i in matches]) + " )" +  '\033[0m' + '\033[0m')
        for idx, match in enumerate(matches):
            print(f"Match {idx+1}: {doc[match[1]: match[2]]} ({match[1]}, {match[2]})")
    else:
        print('\033[91m' + f"[{doc_index+1}] No match found for: {doc.text}" + '\033[0m') 

In [12]:
test_cases_aux_be = [
# WOULD BE
    # NICE
    "When clicking outside of a card on a whiteboard, it would be nice if any selected text could be automatically de-selected",
    "Inside a whiteboard, if I want to backlink a new card, it would be nice to have the card created and added to the current whiteboard.",
    "When scrolling on a card it would be nice if the title would jump to the top (where the buttons are located).",
    "It would be nice to be able to zoom on pictures just like we can do it  natively in MAC OS.",
    "Also it would be nice to always be able to see which cards are scrollable (right now the scrollbar only appears after clicking on the cards).",
    "It would be nice to have a filter in the left nav called something like ToDo and all it does is display cards that have an unfinished task in it.",
    "It would be nice to see them (with appearance different from normal text) next to the corresponding paragraph.",
    "That would be nice to be able to  filter cards by their respective color in the card library",
    "It would be nice if Heptabase supported it natively.",
    "What would be nice to have in Hepta is more options for visual variety.",

    # GREAT
    "It would be great if the different levels of ordered list would have different options, like 1. and then a. then i., like in Notion",
    "It would be great if `-` corresponded to en dashes (–) rather than em dashes (—) and then `--` corresponded to em dashes",
    "It would also be great to be able to copy a link to a card from the card itself (as opposed to relying on search)",
    "As Im starting to use Hepta for some projects (personal and professional), it **would be great to use Hepta truly in an offline mode without the need for Cloud Sync.",
    "It should be great if we can embedded website in cards.",
    "More different arrow styles would be great.",
    "It would be so great to have an option to create links to yet not existing notes and to see list of notes that should be created ranged by number of mentions",
    "Not sure if it’s suggested before but it would be great if there’s some sort of manual grouping that can be implemented under working tabs to sort opened tabs.",
    "On a similar note, would be great to be able to split cards by text blocks (so each text block turns into a separate card on the current board)",

    # AWESOME
    "Handwriting recognition in cards would be awesome as well",

    # HELPFUL
    "It would be helpful to also be able to bring up the right sidebar that looks just like the one in a whiteboard when in the tab view, and be able to select cards to open as additional tabs",
    "I usually dump reading notes in my daily note in outliner form and it would be super helpful to filter the content at block granularity during the review phase when I organise the blocks into cards on the whiteboard.",
    "I think it would be helpful for it remember that I selected the left alignment and paste my images following that alignment",
    "Hello-  I think a wider scroll bar in the card window would be helpful",

    # USEFUL
    "but yes, it would be useful to fold whiteboards and sub-whiteboards like toggle lists",
    "It would be useful to add another type of grey-background \"Block\" (neither code nor math) containing pure text / linked text.",

    # TERRIFIC
    "but it would be terrific if Hepta would allow the same note taking flexibility of Margin Note for PDFs",

# WILL BE
    # COOL
    "Yes i want calendar system too and if can link with google calendar will be cool",

    # GREAT
    "That will be great if we have native support so that I don't have to manually adjust visible part of Carb to match the divider line.",

    # USEFUL
    "I think something small that will be useful is keeping memory of the alignment of the images you paste onto hepta.",

# IT'D BE
    "It'd be great",
    "It'd be so great"
]

In [13]:
test_cases_aux_desire = [
    "I still want to be able to",
    "“When I want to perform an action to internal and external links.",
    "I desperate want an iPhone app so I can at access, if not add, notes and whiteboards on my phon",
    "For my work on a specific machine, I would want this to stay local and offline so that I can keep confidential information there and not up on the cloud",
    "When in a whiteboard I would like an easy way to centre on the active card and zoom in towards it",
    "I would like to recommend that the new keyboard shortcut introduced in v0.172.1 for Windows should be changed to Ctrl + [ or ",
    "While writing an article or essay, I would like to include footnotes or references for each paragraph",
    "as my map grows with more and more whiteboards, I would like to be able to select multiple boards (mouse drag, or shift click each one), and move them around..",
    "As an user I would like to be able to DRAG card references as new Blocks",
    "I would like to be able to reference not only a card but a whiteboard",
    "As a small feature request: I would like to be able to filter by color, as complexity grows, I feel color coding cards/boards will become more meaningful (example: green is 'done', yellow is 'doing'",
    "I'd like to drag a picture (or text)from another app or Finder into a whiteboard and create automatically a card...",
    "I'd like to be able to export selected cards in the order I want as a rich text document (one document - not multiple separate files).",
    "I wish I could assign colors to tags",
    "I wish there were shortcuts to turn a block into different List items faster.",
    "I feel like this might be requested before - but would love to have some form of indicator (even a mini one) to tell that a card on whiteboard has X number of backlink",
    "I would love to see dynamic URI linking as an option. Like if I drop an email or finder file having the option to link rather than uploa",
    "I would love to have global shortcut (that works outside of Hepta. Mem and Things are doing create job with this) and browser extension for capturing content",
    "Would love to be able to download or access a 'Uploaded' folder with all my content",
    "Its a fairly minor detail but I'd love to be able to able to change this (image 1) to this (image 2) without having to move the card",
    "Hello I’d like to be able to scroll longer notes when hovering the mouse cursor over the",
]


In [14]:
could_prounoun_test_cases = [
"*If can link with Google Calendar",
"“If we can be able to change the different highlight colors.",
"*Yes i want calendar system too and if can link with google calendar will be cool”",
"*We could also have a **Pure Text** Skin for a card (or maybe not, that's the devs' decision) and have the possibility to format it, examples: greater font size, different text color, whatever",
"*Maybe we can have something similar?",
"*Just thinking about it, a different 'Skin' for a card could be: pure image",
"*sometimes, it could be useful to have the possibility to just visually group some related cards",
"*I was wondering if there could be an option to remove the text from the original card altogether",
"*That's probably easy to implement and could be very useful.",
"*When you create connection between to boxes, could you add the ability to put a description on each end of the connection as well.",
"*Could you add ABAP syntax in code block?",
"*Please could you put a word count on the card info.",
"Could you add 'system appearance' in addition to 'dark' and 'light' modes",
"*Can we add map groups to collapse maps? Map groups can also nest sub-map groups.",
"*Can there be a card preview when adding a link to a card (similar to how card previews show when searching via Ctrl+O and Ctrl+P)?",
"When dragging a block out of a card to create a new card, could there be an option to NOT create a link to that new card",
"This could be a helpful feature",
"This could be useful"
]

In [15]:
possibility_test_cases = [
    "“*Will it be possible to ad attachments soon?”*",
    "*Would it be possible to get tags visible within white boards on the individual cards?",
    "“*Would it be possible to show a nested whiteboard”",
    "“*I may have missed something, but is it possible to not remove blocks from a card when dragging them to another card?”",
    "“*Is it possible to have Heptabase work with Hook? For the record, hook is an amazing application that provides system wide linking/backlinking features.”",
    "*Is it possible to somehow highlight the text to which you click on the link from Card Info?",
    "“*Hi all, quick silly request: is it possible to see the journal cards in the timeline or 'all cards' preview?”",
    "“*…Another useful visual possibility would be…”",
    "*“- Possibility to add **Pure Images*",
    "Possibility to give **Different Skins** to the car",
    "Possibility for more **Text Formatting”**",
    "“*Possibility to select multiple connections with the intent of either: change the description and make it equal for all the selected connection, delete the, (in the future) To possibly apply other kind of changes if future releases will allow them (example: change of width of the arrow or other kinds)”",
]

In [16]:
suggestion_test_cases = [
    "A few suggestions",
    "My proposal is a new feature",
    "I suggest that we integrate this new feature",
    "I have several ideas that I was hoping I could hear your thoughts on",
    "The proposal is about the following",
    "I propose to take into consideration the idea of implementing in  Heptabase a module for spaced repetition using the Ebbinghaus forgetting curve technology",
    "Several suggestions to reduce typing frictions",
    "A couple suggestions that I don't think have been talked about for the journal.",
    "I would like to recommend that the new keyboard shortcut introduced in v0.172.1 for Windows should be changed to Ctrl + [ or ]",
]

In [17]:
should_test_cases = [
    'The mention itself should be highlighted in color.',
    "It's intuitive as it should be",
    "the new keyboard shortcut introduced in v0.172.1 for Windows should be changed to Ctrl + [ or ]",
    "and to see list of notes that should be created ranged by number of mentions",
    "but I should be able to sort the cards manually as I need, which I believe is not possible now.",
    "That really should change",
]

In [18]:
feature_request_cases = [
    "Feature Request: Be able to see the Card Info before importing them, right from the Card Library list of cards.",
    "feature-request On a Whiteboard, after opening the Card Library on the right sidebar to import them. Sometimes you are not 100% sure which card needs to be imported because there could be a few similar cards that perhaps have small differences just in the content, or the tags, or in which whiteboards they are in.",
    "Feature Request for text editor (minor priority) Scenario:  When I want to perform an action to internal and external links. As-is: -  Use mouse to select precisely the entire text, (2) Left-click/Ctrl+K on 'link' menu, (3) Perform remove/copy action. - If I want to remove a lengthy hyperlink text, I have to select precisely the entire text area. Sometimes with multiple failed attempt. Wish: - When we right-click on the text (no matter where), show the link menu Benefit: Easier operation to remove/copy a lengthy hyperlink text "
]

In [19]:
hope_cases = [
    "Hope there will be an feature 'save multiple working tabs' as a group (which could be re-open in the future) 1",
    "Hope that we can improve the user experience for whiteboards",
    "I hope we can fix this!",
    "I hope you will implement this soon",
]

In [20]:
what_if_cases = [
    "What if we could see the full content of a child whiteboard from a parent whiteboard?"
]

In [21]:
plan_cases = [
    "Hi, do you plan to add images to be displayed right into the title of a card?",
    "Do you plan to add this feature soon?",
    "Do we plan to add this feature soon?",
    "I know there have been a lot of requests for a Readwise Integration with Heptabase but I noticed it’s not on the (public) roadmap. Are there any plans for this?",
    "Hi, are there any plans to implement 'unlinked mentions' in the card info part?",
    "Follow up on this topic, do we have plan to make visually difference on card link and http links?",
    "It is planned to add an in-app browser?",
    "Hi @Alan Chan , it is planned to allow opening multiple windows of the app? "
]

In [22]:
way_to_cases = [
"Is there a way to add this new feature",
"I do like the interface of arranging by whiteboards, but overtime, it can look a bit displeasing to the eyes, so if there is a way to maybe beautify the interfaces a little more? especially the map session, maybe with images or gallery view / other views etc",
"“Is there, or will there be, a way to do a 'portable installation' so Heptabase can be used on Windows computers by users who don't have administrator rights and therefore can't install .exe files?”",
]

In [23]:
experimental_test_cases = [
    # "Can we consider adding this feature?",
    # "This might be a useful feature",
    # "This could be a really useful faeture",
    # "This can be helpful",
    # "This can be very helpful",
    # "This might be very helpful",
    # "This might be a very useful feature",
    # "This would be a cool tool",
    # "This might be a cool tool",
    # "This would be a major plus",
    # "This might be a major plus",
    # "I should be able to sort the cards manually as I need, which I believe is not possible now.",
    # "It should be possible to add this to the existing feature set",
    # "It could be helpful to add this feature",
    # "It can be helpful to add this feature",
    # "It might be helpful to add this feature",
    # "It might be a good idea to add this feature",
    # "I don't think that's a good idea",
    # "This could be a helpful addition to our language",
    'Is there a way to maybe beautify the interface a bit more?'

]

In [24]:
exception_cases = [
    "This is not a good idea",
    "Your suggestions are impossible to implement",
    "This could be a bad idea",
    "Your approach is awful",
    "Your ideas are awful",
    "These what if's don't add anything to the discussion",
    "It is planned to add an in-app browser?",
]

In [25]:
for case in test_cases_aux_be:
    test_string(case)

No match found for: When clicking outside of a card on a whiteboard, it would be nice if any selected text could be automatically de-selected
Match Found for: Inside a whiteboard, if I want to backlink a new card, it would be nice to have the card created and added to the current whiteboard. ( PronounDesire )
Match 1: I want (5, 7)
No match found for: When scrolling on a card it would be nice if the title would jump to the top (where the buttons are located).
No match found for: It would be nice to be able to zoom on pictures just like we can do it  natively in MAC OS.
No match found for: Also it would be nice to always be able to see which cards are scrollable (right now the scrollbar only appears after clicking on the cards).
No match found for: It would be nice to have a filter in the left nav called something like ToDo and all it does is display cards that have an unfinished task in it.
No match found for: It would be nice to see them (with appearance different from normal text) ne

In [26]:
for case in test_cases_aux_desire:
    test_string(case)

No match found for: I still want to be able to
Match Found for: “When I want to perform an action to internal and external links. ( PronounDesire )
Match 1: I want (2, 4)
No match found for: I desperate want an iPhone app so I can at access, if not add, notes and whiteboards on my phon
Match Found for: For my work on a specific machine, I would want this to stay local and offline so that I can keep confidential information there and not up on the cloud ( AuxDesire )
Match 1: would want (9, 11)
Match Found for: When in a whiteboard I would like an easy way to centre on the active card and zoom in towards it ( AuxDesire )
Match 1: would like (5, 7)
Match Found for: I would like to recommend that the new keyboard shortcut introduced in v0.172.1 for Windows should be changed to Ctrl + [ or  ( AuxDesire, ShouldVerb )
Match 1: would like (1, 3)
Match 2: should be changed (15, 18)
Match Found for: While writing an article or essay, I would like to include footnotes or references for each para

In [27]:
for case in could_prounoun_test_cases:
    test_string(case)

No match found for: *If can link with Google Calendar
No match found for: “If we can be able to change the different highlight colors.
Match Found for: *Yes i want calendar system too and if can link with google calendar will be cool” ( PronounDesire )
Match 1: i want (2, 4)
Match Found for: *We could also have a **Pure Text** Skin for a card (or maybe not, that's the devs' decision) and have the possibility to format it, examples: greater font size, different text color, whatever ( PossibilityVerb )
Match 1: possibility to format (31, 34)
No match found for: *Maybe we can have something similar?
No match found for: *Just thinking about it, a different 'Skin' for a card could be: pure image
No match found for: *sometimes, it could be useful to have the possibility to just visually group some related cards
No match found for: *I was wondering if there could be an option to remove the text from the original card altogether
No match found for: *That's probably easy to implement and could 

In [28]:
for case in possibility_test_cases:
    test_string(case)

Match Found for: “*Will it be possible to ad attachments soon?”* ( AuxPossibility, PossibilityAdposition )
Match 1: Will it be possible (2, 6)
Match 2: possible to (5, 7)
Match Found for: *Would it be possible to get tags visible within white boards on the individual cards? ( AuxPossibility, PossibilityVerb )
Match 1: Would it be possible (1, 5)
Match 2: possible to get (4, 7)
Match Found for: “*Would it be possible to show a nested whiteboard” ( AuxPossibility, PossibilityVerb )
Match 1: Would it be possible (2, 6)
Match 2: possible to show (5, 8)
Match Found for: “*I may have missed something, but is it possible to not remove blocks from a card when dragging them to another card?” ( AuxPossibility )
Match 1: is it possible (9, 12)
Match Found for: “*Is it possible to have Heptabase work with Hook? For the record, hook is an amazing application that provides system wide linking/backlinking features.” ( AuxPossibility, PossibilityVerb )
Match 1: Is it possible (2, 5)
Match 2: possible 

In [29]:
for case in suggestion_test_cases:
    test_string(case)

Match Found for: A few suggestions ( DetNounSuggest, DetNounSuggest )
Match 1: A few suggestions (0, 3)
Match 2: few suggestions (1, 3)
Match Found for: My proposal is a new feature ( PronounSuggestionAux )
Match 1: My proposal is (0, 3)
Match Found for: I suggest that we integrate this new feature ( PronounSuggest )
Match 1: I suggest (0, 2)
Match Found for: I have several ideas that I was hoping I could hear your thoughts on ( DetNounSuggest )
Match 1: several ideas (2, 4)
Match Found for: The proposal is about the following ( PronounSuggestionAux )
Match 1: The proposal is (0, 3)
Match Found for: I propose to take into consideration the idea of implementing in  Heptabase a module for spaced repetition using the Ebbinghaus forgetting curve technology ( PronounSuggest )
Match 1: I propose (0, 2)
Match Found for: Several suggestions to reduce typing frictions ( DetNounSuggest )
Match 1: Several suggestions (0, 2)
Match Found for: A couple suggestions that I don't think have been talked

In [30]:
for case in should_test_cases:
    test_string(case)

Match Found for: The mention itself should be highlighted in color. ( ShouldVerb )
Match 1: should be highlighted (3, 6)
No match found for: It's intuitive as it should be
Match Found for: the new keyboard shortcut introduced in v0.172.1 for Windows should be changed to Ctrl + [ or ] ( ShouldVerb )
Match 1: should be changed (9, 12)
Match Found for: and to see list of notes that should be created ranged by number of mentions ( ShouldVerb )
Match 1: should be created (7, 10)
Match Found for: but I should be able to sort the cards manually as I need, which I believe is not possible now. ( ShouldVerb )
Match 1: should be able (2, 5)
Match Found for: That really should change ( ShouldVerb )
Match 1: should change (2, 4)


In [31]:
for case in feature_request_cases:
    test_string(case)

Match Found for: Feature Request: Be able to see the Card Info before importing them, right from the Card Library list of cards. ( FeatureRequestPunct )
Match 1: Request: (1, 3)
No match found for: feature-request On a Whiteboard, after opening the Card Library on the right sidebar to import them. Sometimes you are not 100% sure which card needs to be imported because there could be a few similar cards that perhaps have small differences just in the content, or the tags, or in which whiteboards they are in.
Match Found for: Feature Request for text editor (minor priority) Scenario:  When I want to perform an action to internal and external links. As-is: -  Use mouse to select precisely the entire text, (2) Left-click/Ctrl+K on 'link' menu, (3) Perform remove/copy action. - If I want to remove a lengthy hyperlink text, I have to select precisely the entire text area. Sometimes with multiple failed attempt. Wish: - When we right-click on the text (no matter where), show the link menu Ben

In [32]:
for case in hope_cases:
    test_string(case)

Match Found for: Hope there will be an feature 'save multiple working tabs' as a group (which could be re-open in the future) 1 ( HopeRequest )
Match 1: Hope there (0, 2)
Match Found for: Hope that we can improve the user experience for whiteboards ( HopeRequest )
Match 1: Hope that (0, 2)
Match Found for: I hope we can fix this! ( HopeRequest )
Match 1: hope we (1, 3)
Match Found for: I hope you will implement this soon ( HopeRequest )
Match 1: hope you (1, 3)


In [33]:
for case in what_if_cases:
    test_string(case)

Match Found for: What if we could see the full content of a child whiteboard from a parent whiteboard? ( WhatIf )
Match 1: What if (0, 2)


In [34]:
for case in plan_cases:
    test_string(case)

Match Found for: Hi, do you plan to add images to be displayed right into the title of a card? ( PlannedTo )
Match 1: do you plan to (2, 6)
Match Found for: Do you plan to add this feature soon? ( PlannedTo )
Match 1: Do you plan to (0, 4)
Match Found for: Do we plan to add this feature soon? ( PlannedTo )
Match 1: Do we plan to (0, 4)
Match Found for: I know there have been a lot of requests for a Readwise Integration with Heptabase but I noticed it’s not on the (public) roadmap. Are there any plans for this? ( PlannedTo )
Match 1: Are there any plans for (28, 33)
Match Found for: Hi, are there any plans to implement 'unlinked mentions' in the card info part? ( PlannedTo )
Match 1: are there any plans to (2, 7)
No match found for: Follow up on this topic, do we have plan to make visually difference on card link and http links?
No match found for: It is planned to add an in-app browser?
No match found for: Hi @Alan Chan , it is planned to allow opening multiple windows of the app? 


In [35]:
for case in way_to_cases:
    test_string(case)

Match Found for: Is there a way to add this new feature ( WayTo )
Match 1: Is there a way to (0, 5)
No match found for: I do like the interface of arranging by whiteboards, but overtime, it can look a bit displeasing to the eyes, so if there is a way to maybe beautify the interfaces a little more? especially the map session, maybe with images or gallery view / other views etc
No match found for: “Is there, or will there be, a way to do a 'portable installation' so Heptabase can be used on Windows computers by users who don't have administrator rights and therefore can't install .exe files?”


In [36]:
for case in experimental_test_cases:
    test_string(case)

Match Found for: Is there a way to maybe beautify the interface a bit more? ( WayTo )
Match 1: Is there a way to (0, 5)


In [37]:
for case in exception_cases:
    test_string(case)

Match Found for: This is not a good idea ( DetNounSuggest, DetNounSuggest )
Match 1: a good idea (3, 6)
Match 2: good idea (4, 6)
Match Found for: Your suggestions are impossible to implement ( PronounSuggestionAux )
Match 1: Your suggestions are (0, 3)
Match Found for: This could be a bad idea ( DetNounSuggest, DetNounSuggest )
Match 1: a bad idea (3, 6)
Match 2: bad idea (4, 6)
No match found for: Your approach is awful
Match Found for: Your ideas are awful ( PronounSuggestionAux )
Match 1: Your ideas are (0, 3)
Match Found for: These what if's don't add anything to the discussion ( WhatIf )
Match 1: what if (1, 3)
No match found for: It is planned to add an in-app browser?


## Document Processing

In [38]:
def find_matches(df):
    docs = list(nlp.pipe(df.content))
    for idx, doc in enumerate(docs):
        test_doc(idx, doc)
            

In [39]:
find_matches(df)

[1] No match found for: !history
[2] No match found for: Thanks for the feedback! I'll discuss it with the team
[3] Match Found for: a couple feature requests:


it would be fantastic if whiteboards could be collapsed / folded to show only their title, the same way cards can!


it would be nice to be able to set the size of a collapsed card's title.
-- often I only want the title of a card showing, but for readability I wish I could set the text size to H1 or H2 size, for example. the collapsed card's text is very small for me currently. I get around this by using the card unfolded and resizing the card to hide the rest of the text, but I have to add a block with a period to get an even margin of whitespace around the title (see screenshot - on the 3rd card, the first line of body text is visible and distracting, compared to the other two where I "fake" a full margin and all attention goes to the title). ( PronounDesire, PronounDesire )
Match 1: I only want (51, 54)
Match 2: I wish (64

## Exploratory Analysis

In [95]:
matcherExploration = Matcher(nlp.vocab)

In [116]:
auxBeExploration = [
    [{"POS": "AUX"}, {"POS": "ADV", "OP": "*"}, {"LOWER": "be"}, {"POS": "ADV", "OP": "?"}], 
]

matcherExploration.add("AuxBeExploration", auxBeExploration)

In [146]:
depMatcher = DependencyMatcher(nlp.vocab)
pattern = [
    {
        "RIGHT_ID": "be",
        "RIGHT_ATTRS": {"LOWER": "be"}
    },
    {
        "LEFT_ID": "be",
        "REL_OP": ">",
        "RIGHT_ID": "be_aux",
        "RIGHT_ATTRS": {"DEP": "aux"}
        
    },
    {
        "LEFT_ID": "be",
        "REL_OP": ">",
        "RIGHT_ID": "be_adjective",
        "RIGHT_ATTRS": {"DEP": "acomp", "POS": "ADJ"}
    }
]

depMatcher.add("AuxBeExploration", [pattern])

In [172]:
def test_be_aux(doc):
    matches = depMatcher(doc)
    if len(matches) != 0:
        return [doc[i[1][-1]] for i in matches]
    else:
        return None

In [173]:
def find_exploratory_aux_matches(df):
    aux_words = []
    docs = list(nlp.pipe(df.content))
    for doc in docs:
        if len(matcherExploration(doc)) != 0:
            response = test_be_aux(doc)
            if response is not None:
                aux_words.append(response)
    return aux_words

In [174]:
aux_words = find_exploratory_aux_matches(df)

In [ ]:
aux_words[:20]

In [175]:
# need to take the string of the item because the metadata from the nlp is still invisibly preserved
aux_set = set([str(item).lower() for sublist in aux_words for item in sublist])

In [ ]:
aux_set

#### Pronoun Desire Test

In [128]:
pronounDesireExpMatcher = Matcher(nlp.vocab)

In [131]:
doc = nlp("I really want this feature")
doc[1].pos_

'ADV'

In [132]:
pronounDesireExp = [
    [{"POS": "PRON"}, {"POS": "ADV", "OP": "*"}, {"LOWER": {"IN": ['want', 'wish']}} ]
]

pronounDesireExpMatcher.remove("PronounDesireExp")
pronounDesireExpMatcher.add("PronounDesireExp", pronounDesireExp)

In [133]:
match_list = []
docs = list(nlp.pipe(df.content))
for doc in docs:
    match = pronounDesireExpMatcher(doc)
    if len(match) != 0:
        for val in match:
            match_list.append(doc[val[1]:val[2]])            

In [135]:
set([str(i) for i in match_list])

{'I just want',
 'I only want',
 'I still want',
 'I want',
 'I wish',
 'We want',
 'i later want',
 'i often want',
 'i only want',
 'i want',
 'i wish',
 'they want',
 'we actually want',
 'we just want',
 'we want',
 'who want',
 'you eventually want',
 'you somehow want',
 'you want'}

## Test Area

In [31]:
doc = nlp("will be great")
doc2 = nlp("It would be great")

In [32]:
print(doc[0])
print(doc[0].tag_)
print(doc[0].pos_)

print('\n')
print(doc[1])
print(doc[1].tag_)
print(doc[1].pos_)

print("#########")
print(doc2[0])
print(doc2[0].tag_)
print(doc2[0].pos_)

print('\n')
print(doc2[1])
print(doc2[1].tag_)
print(doc2[1].pos_)


will
MD
AUX


be
VB
AUX
#########
It
PRP
PRON


would
MD
AUX


In [325]:
doc = nlp("Is there a way to")
doc2 = nlp("Do you plan to")
doc3 = nlp("Is there a plan to")
doc4 = nlp("Are there any plans for")
doc5 = nlp("Do we plan to")

    # "Feature Request: Be able to see the Card Info before importing them, right from the Card Library list of cards.",
    # "feature-request On a Whiteboard, after opening the Card Library on the right sidebar to import them. Sometimes you are not 100% sure which card needs to be imported because there could be a few similar cards that perhaps have small differences just in the content, or the tags, or in which whiteboards they are in.",
    # "Feature Request for text editor (minor priority) Scenario:  When I want to perform an action to internal and external links. As-is: -  Use mouse to select precisely the entire text, (2) Left-click/Ctrl+K on 'link' menu, (3) Perform remove/copy action. - If I want to remove a lengthy hyperlink text, I have to select precisely the entire text area. Sometimes with multiple failed attempt. Wish: - When we right-click on the text (no matter where), show the link menu Benefit: Easier operation to remove/copy a lengthy hyperlink text "


options = {
    "fine_grained": False
}

displacy.render(doc, options=options)
displacy.render(doc2, options=options)
displacy.render(doc3, options=options)
displacy.render(doc4, options=options)
displacy.render(doc5, options=options)


In [25]:
print(doc[1].tag_)
print(doc2[1].tag_)

doc2[1]

VBP
MD


would